In [1]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import numpy as np
import itertools
from sys import stdout


### Creates a coarse-grained (CG) element to be used in simulations.

This is an interesting feature that you'd only encounter in coarse-grained simulations. In CG, of course, you'd NEED to deifne the "atomic number", mass, charge etc of the CG center that you are creating. That's what we do next.

In [2]:
cgElement = app.Element(number=1000, name='CG-element', symbol='CG', mass=120)

'''
#Given a set of masses, create a set of elements
if 'elements_initialized' not in globals():
    elements = []
    for i, mass in enumerate(MASS):
        element = app.Element(number=1000 + i, name=f'C{i+1}', symbol=f'C{i+1}', mass=mass)
        elements.append(element)
    elements_initialized = True
'''

"\n#Given a set of masses, create a set of elements\nif 'elements_initialized' not in globals():\n    elements = []\n    for i, mass in enumerate(MASS):\n        element = app.Element(number=1000 + i, name=f'C{i+1}', symbol=f'C{i+1}', mass=mass)\n        elements.append(element)\n    elements_initialized = True\n"

In [3]:
# Make an empty topology
topology = app.Topology()

### The System

In this notebook, we will create a simulation box with 100 polymer chains, with 10 beads each. 

In [4]:
# Number of polymer chains
M = 100
# Number of atoms in each chain
N = 10

# Add each chain to the topology
for m in range(M):
    chain = topology.addChain()
    atom1 = topology.addAtom(name="CG-bead", element=cgElement, residue=topology.addResidue(name="CG-residue", chain=chain))
    
    for i in range(1, N):
        atom2 = topology.addAtom(name="CG-bead", element=cgElement, residue=topology.addResidue(name="CG-residue", chain=chain))
        topology.addBond(atom1, atom2)
        atom1 = atom2

# Check the topology
print(topology)


<Topology; 100 chains, 1000 residues, 1000 atoms, 900 bonds>


Now, we place the beads in a grid. Distance between beads in a single polymer is 0.38 nm

In [5]:
# Initialize an empty list to store positions
positions = []
# Loop over each polymer chain
for m in range(M):
    # Calculate the initial position for the first bead in the chain
    x0 = np.array(((m % 10) * 1.0, (m // 10) * 1.0, 0))
    positions.append(x0)    
    # Loop over the remaining beads in the chain
    for i in range(1, N):
        # Calculate the position for the next bead in the chain
        xi = positions[-1] + np.array((0, 0, 0.38))
        positions.append(xi)

# Convert the list of positions into an OpenMM Quantity with units of nanometers
positions = positions * unit.nanometer
# Ensure the number of positions matches the number of atoms in the topology
assert len(positions) == topology.getNumAtoms()
# Set the periodic box vectors to create a cubic box with a length of 11 nm
topology.setPeriodicBoxVectors(np.eye(3) * 11.0 * unit.nanometers)


In [6]:
# output the initial configuration. Save it to a file
with open('initial_config.pdb','w') as f:
    app.PDBFile.writeFile(topology, positions, f)


At this point, we now move to OpenMM, and prepare the simulation.

In [7]:
# create the system and add the particles to it
system = mm.System()
system.setDefaultPeriodicBoxVectors(*topology.getPeriodicBoxVectors())
for atom in topology.atoms():
    system.addParticle(atom.element.mass)

### Defining the forcefield.
*Bonds*: Equilibrium bond lengths and spring constants at 0.38 nm and 1000 kJ/mol/nm respectively 

*Angles*: 0.5*k*(cos(angle)-cos(theta0))^2 ; k=10 kJ/mol/rad^2; theta0=180 deg  

*Non-bonded*: LJ potential - sigma=0.5 nm; epsilon - 1 kJ/mol

In [8]:
harmonic_bond_force = mm.HarmonicBondForce()

# Add each bond to the force from the topology
for bond in topology.bonds():
    harmonic_bond_force.addBond(bond.atom1.index, bond.atom2.index, 0.38, 1000)

'''
custom_angle = mm.CustomAngleForce("0.5*k*(cos(theta)-cos(theta0))^2")
custom_angle.addPerAngleParameter('k')
custom_angle.addPerAngleParameter('theta0')
# Loop through all chains and assign angles for each three bonded atoms
for chain in topology.chains():
    atoms = list(chain.atoms())
    for i in range(len(atoms) - 2):
        custom_angle.addAngle(atoms[i].index, atoms[i+1].index, atoms[i+2].index, [10, 3.14159])
'''
        
# Define a Lennard-Jones potential
expression = '4*epsilon*((sigma/r)^12-(sigma/r)^6);'\
            + ' sigma=0.5*(sigma1+sigma2);'\
            + ' epsilon=sqrt(epsilon1*epsilon2)'

custom_nb_force = mm.CustomNonbondedForce(expression)

custom_nb_force.addPerParticleParameter('sigma')
custom_nb_force.addPerParticleParameter('epsilon')

# Add the parameters for each particle
for atom in topology.atoms():
    custom_nb_force.addParticle([0.5, 1.0])

# Create exclusions for directly bonded atoms
custom_nb_force.createExclusionsFromBonds([(bond[0].index, bond[1].index) for bond in topology.bonds()], 1)

# Set a cutoff of 1.5nm
custom_nb_force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffPeriodic)
custom_nb_force.setCutoffDistance(1.5*unit.nanometers)

# Add the forces to the system
system.addForce(harmonic_bond_force)
#system.addForce(custom_angle)
system.addForce(custom_nb_force)

1

In [9]:
with open('system1.xml', 'w') as output:
    output.write(mm.XmlSerializer.serialize(system))

In [10]:
#Running the simulation - very similar to proein-in_water simulation
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 0.01/unit.picosecond, 0.010*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)
simulation.context.setPositions(positions)

# setup simulation reporters
# Write the trajectory to a file called 'traj.dcd'
simulation.reporters.append(app.DCDReporter('traj.dcd', 1000, enforcePeriodicBox=False))

# Report information to the screen as the simulation runs
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True, speed=True))


# NVT equilibration
simulation.step(10000)
# Add a barostat
barostatIndex=system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin))
simulation.context.reinitialize(preserveState=True)
# Run NPT equilibration
simulation.step(100000)


# output the equilibrated configuration
with open('equilibrated_config.pdb','w') as f:
    state = simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
    topology.setPeriodicBoxVectors(state.getPeriodicBoxVectors())
    app.PDBFile.writeFile(topology, state.getPositions(), f)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Speed (ns/day)"
1000,-2579.05341758579,193.78898629733848,1331.0,0
2000,-2760.042927503586,224.14985640929433,1331.0,6.56e+03
3000,-2667.2781274318695,231.11849172172523,1331.0,6.56e+03
4000,-2685.999105580151,240.72352964040704,1331.0,6.45e+03
5000,-2778.2200746536255,249.30813780115835,1331.0,6.45e+03
6000,-2669.151108264923,248.5653719458498,1331.0,6.59e+03
7000,-2527.026636123657,254.4083621903991,1331.0,6.71e+03
8000,-2455.047845840454,251.3560011082866,1331.0,6.81e+03
9000,-2418.755630970001,254.17191792212907,1331.0,6.89e+03
10000,-2471.957194328308,259.8185868809086,1331.0,6.94e+03
11000,-2435.6687593460083,261.8869061601297,1252.780305853384,5.79e+03
12000,-2667.216323465109,284.5862897082112,1167.7969485426895,5.25e+03
13000,-2572.6606636047363,276.75451809152014,1075.812794652569,4.63e+03
14000,-2273.3719396591187,266.205423552397,1069.9079544649214,4.3e+03
15000,-2322.160106420517,279.13509638775747

KeyboardInterrupt: 